In [ ]:
import pandas as pd
import numpy as np
from lxml import etree as ET
from lxml.etree import XMLSyntaxError
from pymongo import MongoClient
import os
import codecs
import re

In [ ]:
client = MongoClient('localhost',27017)
db = client['db_data_gov']

def save_mongo(dataframe, collection_name=None):
    result = db[collection_name].insert_many(dataframe.to_dict(orient='records'))
    return result

In [ ]:
def processRootElement(root):
    if root is None:
        raise ValueError("Parâmetro tree não pode ser nulo")
    count = 0
    data = []
    for child in root:
        data.append(dict(child.attrib))
        count += 1
    df = pd.DataFrame(data)
    return df, count

def fromXMLAsStringToDataFrame(filename):
    df = None
    count = 0
    regex = u'[!@#$%¨&*()¬§]' # caracteres especiais a serem eliminados do arquivo
    try :
        xmlfile = codecs.open(filename,mode='r',encoding='utf-16')
        content = xmlfile.read() # estes arquivos geralmente contem todos os registros em uma única linha
        content = re.sub(regex,'',content)
        root = ET.fromstring(content) # A funcao ET.fromstring retorna um objeto do tipo Element que ja e o proprio root Element
        df, count = processRootElement(root) # por isso nao precisamos ja passamos o root Element para a nossa funcao
        return df, count
    except Exception as ERR :
        raise Exception(ERR,'erro em fromXMLAsStringToDataFrame')  

def fromXMLtoDataFrame(filename):
    df = None
    count = 0
    try:
        tree = ET.parse(filename) # A funcao ET.parse retorna um objeto do tipo ElementTree
        df, count = processRootElement(tree.getroot()) # por isso precisamos obter o root Element de tree com .getroot()
        return df, count
    except XMLSyntaxError as ERR :
        print(ERR)
        print('XMLSyntaxError occurred  -> using fromXMLAsStringToDataFrame function to process file: ')
        print(filename)
        df, count = fromXMLAsStringToDataFrame(filename)
        return df, count

In [ ]:
'''Função responsável por garantir as conversões '''
def parserFromLicitacao(df_licitacao):
    casting_dcit = {
        'vlLicitacao' : 'float',
        'dtAbertura' : 'datetime64[ns]',
        'dtEdital' : 'datetime64[ns]',
        'dtEnvio' : 'datetime64[ns]',
    }
    df_licitacao.astype(casting_dcit,copy=False)
    
    return df_licitacao

def parserFromLicitacaoParticipante(df_participante):
    return df_participante

def parserFromLicitacaoVencedor(df_vencedor):
    casting = {
        'dtHomologacao' : 'datetime64[ns]',
        'dtPrazoEntregaPropostaLicitacao' : 'datetime64[ns]',
        'dtValidadeProposta' : 'datetime64[ns]',
        'nrQuantidade' : 'float',
        'nrQuantidadePropostaLicitacao' : 'float',
        'nrQuantidadeVencedorLicitacao' : 'float',
        'vlLicitacaoVencedorLicitacao' : 'float',
        'vlMaximoTotal' : 'float',
        'vlMaximoUnitarioitem' : 'float',
        'vlMinimoTotal' : 'float',
        'vlMinimoUnitarioItem' : 'float',
        'vlPropostaItem' : 'float'
    }
    return df_vencedor.astype(casting,copy=False)

In [ ]:
def determine_file_type(file, f_config=files_config):
    index = file.rfind('_')
    if index < 0 : 
        raise Error('Key error')
    else :
        index += 1
        f_type = file[index:]
        return f_config[f_type]

def files_handler(files = [], saveInMongo = True, saveAsCSV = False):
    verificar = []
    for filename in files :
        config = determine_file_type(filename)
        parser = config['parser']
        try:
            df,qtd = fromXMLtoDataFrame(filename)
            df = parser(df)
            if saveInMongo:
                result = save_mongo(df, collection_name=config['collection'])
                if qtd != len(result.inserted_ids):
                    verificar.append(filename)
            if saveAsCSV :
                i = filename.rfind('/') + 1 # garante que i seja pelo menos zero
                csvname = filename[i:-4] + '.csv' # exclue o .xml do final do file name
                df.to_csv('data/csv/licitacao/'+csvname) # os diretorios data/csv/licitacao ja devem existir
        except Exception as ERR:
            print(ERR)
            print('file name: ',filename)
            raise ERR
    if verificar :
        return verificar
    else :
        return True
        

In [ ]:
diretorio = "data/licitacao/"
files = os.listdir(diretorio)
files = list(map(lambda f : diretorio + f, files))

len(files)

In [ ]:
files_config = {
    'Licitacao.xml' : {
        'parser' : parserFromLicitacao,
        'collection' : 'licitacao'
    },
    'LicitacaoVencedor.xml' : {
        'parser' : parserFromLicitacaoVencedor,
        'collection' : 'licitacaoVencedor'
    },
    'LicitacaoParticipante.xml' : {
        'parser' : parserFromLicitacaoParticipante,
        'collection' : 'licitacaoParticipante '
    }
}

In [ ]:
list(files_config.keys())

In [ ]:
files[20] # exemplo de elemento do array

In [ ]:
files_handler(files, saveInMongo = False, saveAsCSV = True)